In [20]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb
import optuna
import numpy as np

# Set the Optuna logger to output only WARNING and higher levels
optuna.logging.set_verbosity(optuna.logging.WARNING)

sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
dtype_dict = {
    'Date': 'str',
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'Age': 'Int64',
    'BreedName': 'str',
    'DailyYield': 'float',
    'PreviousDailyYield': 'float',
    'DailyYieldChange': 'float',
    'DaysInMilk': 'Int64',
    'YearSeason': 'str',
    'LactationNumber': 'Int64',
    'ExpectedYield': 'float',
    'NormalizedDailyYield': 'float',
    'NormalizedDailyYieldChange': 'float',
    'HeatStress': 'Int64',
    'Temp15Threshold': 'Int64',
    'HW': 'Int64',
    'cum_HW': 'Int64',
    'MeanTemperature': 'float',
    'MeanTHI_adj': 'float'
}

milk_data = pd.read_csv('../Data/MergedData/MilkApproachYieldDataTest.csv', dtype=dtype_dict)
milk_data['Date'] = pd.to_datetime(milk_data['Date'], format='%Y-%m-%d')
milk_data.head(-5)

,Date,FarmName_Pseudo,SE_Number,Age,BreedName,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,DailyYieldChange,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj
0,2022-01-01,a624fb9a,SE-064c0cec-1189,3095,02 SLB,7,191,2022-1,30.77,0.000000,0.000000,35.914865,0.856748,0.000000,0,0,0,0,-3.025000,28.012944
1,2022-01-02,a624fb9a,SE-064c0cec-1189,3096,02 SLB,7,192,2022-1,48.22,30.770000,8.725000,35.799613,1.103224,0.243718,0,0,0,0,-0.279167,32.898193
2,2022-01-03,a624fb9a,SE-064c0cec-1189,3097,02 SLB,7,193,2022-1,30.53,39.495000,-2.988333,35.684360,1.023044,-0.083744,0,0,0,0,2.033333,36.760487
3,2022-01-04,a624fb9a,SE-064c0cec-1189,3098,02 SLB,7,194,2022-1,42.26,36.506667,1.438333,35.569108,1.066796,0.040438,0,0,0,0,0.066667,31.939524
4,2022-01-05,a624fb9a,SE-064c0cec-1189,3099,02 SLB,7,195,2022-1,38.49,37.945000,0.109000,35.453856,1.073339,0.003074,0,0,0,0,-3.700000,26.498206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483097,2023-06-03,f454e660,SE-fcdf259d-0044-0,4150,41 Fjällko,10,347,2023-3,12.67,14.652000,-0.622000,13.608593,1.030966,-0.045706,0,1,0,0,12.666667,53.132530
483098,2023-06-04,f454e660,SE-fcdf259d-0044-0,4151,41 Fjällko,10,348,2023-3,22.31,14.030000,0.954000,13.516773,1.108549,0.070579,0,1,0,0,13.079167,56.726870
483099,2023-06-05,f454e660,SE-fcdf259d-0044-0,4152,41 Fjällko,10,349,2023-3,12.84,14.984000,-0.092000,13.424952,1.109278,-0.006853,0,1,0,0,14.237500,58.482418
483100,2023-06-06,f454e660,SE-fcdf259d-0044-0,4153,41 Fjällko,10,350,2023-3,9.47,14.892000,-0.284000,13.333131,1.095617,-0.021300,0,1,0,0,15.345833,60.546358


In [22]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['FarmName_Pseudo', 'FarmHeatStressProduction'])

# Specify the farm ID for analysis
farm_id = 'a624fb9a'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.000214161560042
Standard Deviation of NormalizedDailyYield: 0.10190530213781133
Variance of NormalizedDailyYield: 0.010384690603798616

Selected features: ['HeatStress']
Best parameters: {'lambda': 0.501463959804858, 'alpha': 0.9963887725222779, 'subsample': 0.5460620581051139, 'colsample_bytree': 0.7135863761558365, 'learning_rate': 0.07249799419761006, 'n_estimators': 702, 'max_depth': 14}
Estimated average production during heat stress: 0.9099
Estimated average production without heat stress: 1.0087
Reduction in milk production during heat stress: -0.0988


,FarmName_Pseudo,FarmHeatStressProduction
0,a624fb9a,-0.098792


In [23]:
# Specify the farm ID for analysis
farm_id = '5c06d92d'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 0.9998734816219872
Standard Deviation of NormalizedDailyYield: 0.11010619232060219
Variance of NormalizedDailyYield: 0.012123373587341436

Selected features: ['HeatStress']
Best parameters: {'lambda': 0.02619616199338888, 'alpha': 0.0009975569593760067, 'subsample': 0.795842938387751, 'colsample_bytree': 0.5918148240854036, 'learning_rate': 0.09965967082967458, 'n_estimators': 652, 'max_depth': 1}
Estimated average production during heat stress: 0.9299
Estimated average production without heat stress: 1.0037
Reduction in milk production during heat stress: -0.0738


,FarmName_Pseudo,FarmHeatStressProduction
0,a624fb9a,-0.098792
1,5c06d92d,-0.073769


In [24]:
# Specify the farm ID for analysis
farm_id = '752efd72'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0000642245718283
Standard Deviation of NormalizedDailyYield: 0.0731526227092293
Variance of NormalizedDailyYield: 0.0053513062092388495

Selected features: ['HeatStress']
Best parameters: {'lambda': 0.002115553931938036, 'alpha': 9.728052571854744e-06, 'subsample': 0.5004055506891417, 'colsample_bytree': 0.9309193333616297, 'learning_rate': 0.08227758756891332, 'n_estimators': 182, 'max_depth': 3}
Estimated average production during heat stress: 0.9265
Estimated average production without heat stress: 1.0060
Reduction in milk production during heat stress: -0.0795


,FarmName_Pseudo,FarmHeatStressProduction
0,a624fb9a,-0.098792
1,5c06d92d,-0.073769
2,752efd72,-0.079519


In [25]:
# Specify the farm ID for analysis
farm_id = 'f454e660'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 0.9998738797329533
Standard Deviation of NormalizedDailyYield: 0.11483306817532685
Variance of NormalizedDailyYield: 0.013186633546559265

Selected features: ['HeatStress']
Best parameters: {'lambda': 0.010685355851864978, 'alpha': 3.2279448791835197e-06, 'subsample': 0.7193516549004851, 'colsample_bytree': 0.65475320277065, 'learning_rate': 0.030135500328480835, 'n_estimators': 921, 'max_depth': 13}
Estimated average production during heat stress: 0.9527
Estimated average production without heat stress: 1.0010
Reduction in milk production during heat stress: -0.0483


,FarmName_Pseudo,FarmHeatStressProduction
0,a624fb9a,-0.098792
1,5c06d92d,-0.073769
2,752efd72,-0.079519
3,f454e660,-0.048316


In [26]:
# Merge the results_df with milk_data on 'FarmName_Pseudo'
milk_data = milk_data.merge(results_df, on='FarmName_Pseudo', how='left')

# Set 'FarmHeatStressProduction' to NaN where 'HeatStress' is 0
milk_data.loc[milk_data['HeatStress'] == 0, 'FarmHeatStressProduction'] = np.nan
milk_data.head(-5)

,Date,FarmName_Pseudo,SE_Number,Age,BreedName,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,...,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj,FarmHeatStressProduction
0,2022-01-01,a624fb9a,SE-064c0cec-1189,3095,02 SLB,7,191,2022-1,30.77,0.000000,...,35.914865,0.856748,0.000000,0,0,0,0,-3.025000,28.012944,NaN
1,2022-01-02,a624fb9a,SE-064c0cec-1189,3096,02 SLB,7,192,2022-1,48.22,30.770000,...,35.799613,1.103224,0.243718,0,0,0,0,-0.279167,32.898193,NaN
2,2022-01-03,a624fb9a,SE-064c0cec-1189,3097,02 SLB,7,193,2022-1,30.53,39.495000,...,35.684360,1.023044,-0.083744,0,0,0,0,2.033333,36.760487,NaN
3,2022-01-04,a624fb9a,SE-064c0cec-1189,3098,02 SLB,7,194,2022-1,42.26,36.506667,...,35.569108,1.066796,0.040438,0,0,0,0,0.066667,31.939524,NaN
4,2022-01-05,a624fb9a,SE-064c0cec-1189,3099,02 SLB,7,195,2022-1,38.49,37.945000,...,35.453856,1.073339,0.003074,0,0,0,0,-3.700000,26.498206,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483097,2023-06-03,f454e660,SE-fcdf259d-0044-0,4150,41 Fjällko,10,347,2023-3,12.67,14.652000,...,13.608593,1.030966,-0.045706,0,1,0,0,12.666667,53.132530,NaN
483098,2023-06-04,f454e660,SE-fcdf259d-0044-0,4151,41 Fjällko,10,348,2023-3,22.31,14.030000,...,13.516773,1.108549,0.070579,0,1,0,0,13.079167,56.726870,NaN
483099,2023-06-05,f454e660,SE-fcdf259d-0044-0,4152,41 Fjällko,10,349,2023-3,12.84,14.984000,...,13.424952,1.109278,-0.006853,0,1,0,0,14.237500,58.482418,NaN
483100,2023-06-06,f454e660,SE-fcdf259d-0044-0,4153,41 Fjällko,10,350,2023-3,9.47,14.892000,...,13.333131,1.095617,-0.021300,0,1,0,0,15.345833,60.546358,NaN


In [27]:
# Save the new data to a new CSV file called 'XGBMilkFarmYieldData.csv' in same folder
milk_data.to_csv('../Data/MergedData/XGBMilkFarmYieldData.csv', index=False)

### Variables Explanation for `XGBMilkFarmYieldData.csv`

1. **Date**:
   - Description: The date when the milk yield was recorded.
   - Datatype: `datetime`
   - Format: `YYYY-MM-DD`
   - Example: `2022-01-01`

2. **FarmName_Pseudo**:
   - Description: A pseudo-identifier for the farm where the data was collected.
   - Datatype: `str`
   - Example: `a624fb9a`

3. **SE_Number**:
   - Description: A unique identifier for the cow, which has been formatted to include the farm and the animal number.
   - Datatype: `str`
   - Example: `SE-064c0cec-1189`

4. **Age**:
   - Description: The age of the cow in days.
   - Datatype: `Int64`
   - Example: `3095`

5. **BreedName**:
   - Description: The breed name of the cow.
   - Datatype: `str`
   - Example: `02 SLB`

6. **LactationNumber**:
   - Description: The number assigned to the cow's lactation cycle.
   - Datatype: `Int64`
   - Example: `7`

7. **DaysInMilk**:
   - Description: The number of days the cow has been in milk (lactating) at the time of recording.
   - Datatype: `Int64`
   - Example: `191`

8. **YearSeason**:
   - Description: The seasonal period based on the year and the month range.
   - Datatype: `str`
   - Example: `2022-1`
   - YearSeason parameters in yield datasets:
     - 1: Dec-Feb
     - 2: Mar-May
     - 3: Jun-Aug
     - 4: Sep-Nov

9. **DailyYield**:
   - Description: The total amount of milk produced by the cow in a single day.
   - Datatype: `float`
   - Example: `30.77`

10. **PreviousDailyYield**:
    - Description: The total amount of milk produced by the cow on the previous day.
    - Datatype: `float`
    - Example: `0.0`

11. **DailyYieldChange**:
    - Description: The change in daily milk yield from the previous day.
    - Datatype: `float`
    - Example: `0.0`

12. **ExpectedYield**:
    - Description: The expected amount of milk yield based on certain models or predictions.
    - Datatype: `float`
    - Example: `35.914865`

13. **NormalizedDailyYield**:
    - Description: The daily yield normalized to account for various factors.
    - Datatype: `float`
    - Example: `0.856748`

14. **NormalizedDailyYieldChange**:
    - Description: The change in normalized daily yield from the previous day.
    - Datatype: `float`
    - Example: `0.0`

15. **HeatStress**:
    - Description: A binary variable indicating the presence of heat stress on the cow.
    - Datatype: `Int64`
    - Example: `0`

16. **Temp15Threshold**:
    - Description: A binary variable indicating if the temperature exceeded 15 degrees Celsius on the given day.
    - Datatype: `Int64`
    - Example: `0`

17. **HW**:
    - Description: A binary variable indicating the presence of a heatwave on the day.
    - Datatype: `Int64`
    - Example: `0`

18. **cum_HW**:
    - Description: Cumulative number of heatwave days up to the current date.
    - Datatype: `Int64`
    - Example: `0`

19. **MeanTemperature**:
    - Description: The mean temperature recorded on the day.
    - Datatype: `float`
    - Example: `-3.025`

20. **MeanTHI_adj**:
    - Description: The mean adjusted Temperature-Humidity Index for the day.
    - Datatype: `float`
    - Example: `28.012944`

21. **FarmHeatStressMilkProduction**:
    - Description: The relative change in milk production based on farm and heat stress conditions.
    - Datatype: `float`
    - Example: `0.009435`